In [1]:
%matplotlib inline

import sys
sys.path.append('../src/')
import warnings
warnings.filterwarnings('ignore')
from benchmark.tools import *
from benchmark.models import Detector
from benchmark.utils import TsvHandle, run_detection, run_detection_video
from benchmark.stats import calc_precision_recall, calc_ap

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.integrate import simps
plt.rcParams["figure.figsize"] = (15, 10)

import torch
import wider_dataset
import choke_point_dataset

# https://github.com/timesler/facenet-pytorch/
from facenet_pytorch import MTCNN

# MTCNN Image Detection

In [3]:
train, test, validation = tfds.load('wider_dataset', split=['train', 'test', 'validation'])

In [4]:
data = validation

In [5]:
class MtcnnDetector(Detector):
    def __init__(self):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print('Running on device: {}'.format(device))
        
        self.detector = MTCNN(margin=14,
                            factor=0.6,
                            keep_all=True,
                            device=device)
    
    def detect(self, image):
        boxes, confidence = self.detector.detect(image)
        return boxes, confidence

In [6]:
mtcnn = MtcnnDetector()

Running on device: cpu


In [7]:
with TsvHandle('mtcnn_filtered') as tsv:
    run_detection(tsv, data, mtcnn, None, False, False, None, 2000)

KeyboardInterrupt: 

# Results

In [ ]:
path = 'mtcnn-20220418-125039'
path_filtered = 'mtcnn_filtered-20220427-180935'

df = pd.read_csv('./results/{}.tsv'.format(path), sep='\t', header=0, parse_dates=["Timestamp"])
df_filtered = pd.read_csv('./results/{}.tsv'.format(path_filtered), sep='\t', header=0, parse_dates=["Timestamp"])

df_ap = pd.read_csv('./results/{}-ap.tsv'.format(path), sep='\t', header=0)
df_ap = df_ap.sort_values(by=['Confidence'], ignore_index=True, ascending=False)

df_ap_filtered = pd.read_csv('./results/{}-ap.tsv'.format(path_filtered), sep='\t', header=0)
df_ap_filtered = df_ap_filtered.sort_values(by=['Confidence'], ignore_index=True, ascending=False)

df_load = pd.read_csv('./results/{}-load.tsv'.format(path), sep='\t', header=0, parse_dates=["Timestamp"])
df_meta = pd.read_csv('./results/{}-metadata.tsv'.format(path), sep='\t', header=0)

In [ ]:
df_meta

In [ ]:
df_load

In [ ]:
def plot_data_over_time(time, data, label):
    plt.plot(time, data)
    plt.xlabel('Time') 
    plt.ylabel(label)
    plt.grid(True)
    plt.show()

In [ ]:
plot_data_over_time(df_load['Timestamp'], df_load['CPU_Load'], 'CPU Load')

In [ ]:
plot_data_over_time(df_load['Timestamp'], df_load['RAM_Avail'], 'Available memory [GB]')

In [ ]:
sns.FacetGrid(df_load, height=6).map(sns.kdeplot, 'CPU_Load').add_legend()
plt.title('CPU Load Distribution')
plt.xlim(0, 100)
plt.show()

In [ ]:
sns.FacetGrid(df_load, height=6).map(sns.kdeplot, 'RAM_Avail').add_legend()
plt.title('Memory Availability Distribution')
plt.show()

In [ ]:
df

In [ ]:
df.describe()

### Generate Precision/Recall columns

In [ ]:
all_gts = df['Positives'].sum() + df['Negatives'].sum()
df_ap = calc_precision_recall(df_ap, all_gts)

df_ap

In [ ]:
all_gts_filtered = df_filtered['Positives'].sum() + df_filtered['Negatives'].sum()
df_ap_filtered = calc_precision_recall(df_ap_filtered, all_gts_filtered)

df_ap_filtered

In [ ]:
def plot_pr(df_data, title):
    ap = calc_ap(df_data['Precision@25'], df_data['Recall@25'])
    plt.plot(df_data['Recall@25'], df_data['Precision@25'], color='darkcyan', lw=3, label="AP@0.25=%.2f" % ap)
    plt.axvline(x = df_data['Recall@25'][df_data.index[-1]], ymax=df_data['Precision@25'][df_data.index[-1]], color = 'darkcyan')
    
    ap = calc_ap(df_data['Precision@50'], df_data['Recall@50'])
    plt.plot(df_data['Recall@50'], df_data['Precision@50'], color='darkorange', lw=3, label="AP@0.5=%.2f" % ap)
    plt.axvline(x = df_data['Recall@50'][df_data.index[-1]], ymax=df_data['Precision@50'][df_data.index[-1]], color = 'darkorange')
    
    ap = calc_ap(df_data['Precision@75'], df_data['Recall@75'])
    plt.plot(df_data['Recall@75'], df_data['Precision@75'], color='mediumorchid', lw=3, label="AP@0.75=%.2f" % ap)
    plt.axvline(x = df_data['Recall@75'][df_data.index[-1]], ymax=df_data['Precision@75'][df_data.index[-1]], color = 'mediumorchid')
    
    plt.xlim(0, 1.00)
    plt.ylim(0, 1.00)

    plt.title(title)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend(loc="lower right", prop={'size': 16})
    plt.grid(True)

    plt.show()

In [ ]:
plot_pr(df_ap, 'MTCNN Average Precision on filtered WIDER FACE')

In [ ]:
plot_pr(df_ap_filtered, 'MTCNN Average Precision on filtered WIDER FACE')

# MTCNN Video detection

In [2]:
test = tfds.load('choke_point', split=['test'])

In [3]:
data = test[0]

In [4]:
class MtcnnDetector(Detector):
    def __init__(self):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print('Running on device: {}'.format(device))
        
        self.detector = MTCNN(margin=14,
                            factor=0.6,
                            keep_all=True,
                            device=device)
    
    def detect(self, image):
        boxes, confidence = self.detector.detect(image)
        return boxes, confidence
    
mtcnn = MtcnnDetector()

Running on device: cpu


In [5]:
stt_aps, mean_confidences, mean_inference_times = run_detection_video(
    samples=data, 
    detector=mtcnn,
    display_results=False,
    save_anim="mtcnn"
)

In [9]:
print("Mean Video STT-AP: {0:.2f}".format(np.mean(stt_aps)))
print("Mean confidence of boxes in all videos: {:.2f}".format(np.mean(mean_confidences)))
print("Mean inference time of all frames: {:.2f}ms".format(np.mean(mean_inference_times) / 1e+6))

Mean Video STT-AP: 0.69
Mean confidence of boxes in all videos: 1.00
Mean inference time of all frames: 84.42ms
